In [28]:
import pandas as pd
#원본 http://goo.gl/4oDg4Y

### Load Dataset

In [29]:
train = pd.read_csv("data/train.tsv", sep="\t", index_col="PhraseId")

print(train.shape)
train.head()

(156060, 3)


,SentenceId,Phrase,Sentiment
PhraseId,,,
1,1,A series of escapades demonstrating the adage ...,1
2,1,A series of escapades demonstrating the adage ...,2
3,1,A series,2
4,1,A,2
5,1,series,2


In [30]:
test = pd.read_csv("data/test.tsv", sep="\t", index_col="PhraseId")

print(test.shape)
test.head()

(66292, 2)


,SentenceId,Phrase
PhraseId,,
156061,8545,An intermittently pleasing but mostly routine ...
156062,8545,An intermittently pleasing but mostly routine ...
156063,8545,An
156064,8545,intermittently pleasing but mostly routine effort
156065,8545,intermittently pleasing but mostly routine


## Preprocessing

In [31]:
train["Phrase(Origin)"] = train["Phrase"].copy()

print(train.shape)
train[["Phrase", "Phrase(Origin)"]].head()

(156060, 4)


,Phrase,Phrase(Origin)
PhraseId,,
1,A series of escapades demonstrating the adage ...,A series of escapades demonstrating the adage ...
2,A series of escapades demonstrating the adage ...,A series of escapades demonstrating the adage ...
3,A series,A series
4,A,A
5,series,series


In [32]:
test["Phrase(Origin)"] = test["Phrase"].copy()

print(test.shape)
test[["Phrase", "Phrase(Origin)"]].head()

(66292, 3)


,Phrase,Phrase(Origin)
PhraseId,,
156061,An intermittently pleasing but mostly routine ...,An intermittently pleasing but mostly routine ...
156062,An intermittently pleasing but mostly routine ...,An intermittently pleasing but mostly routine ...
156063,An,An
156064,intermittently pleasing but mostly routine effort,intermittently pleasing but mostly routine effort
156065,intermittently pleasing but mostly routine,intermittently pleasing but mostly routine


## (2nd) NLTK

# Stemmer

In [91]:
#nltk
import nltk
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

from nltk.stem.snowball import SnowballStemmer
stemmer= SnowballStemmer('english')
stemmer

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\optim\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\optim\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [97]:
phrase = train.loc[2274]["Phrase"]

In [98]:
#진척도 표시. 뭔가 이상함- 하나로 안나오고 줄줄이. tqdm 자체 에러래. 신경 X
from tqdm import tqdm

def stem_phrase(phrase):
    words = phrase.split(" ")

    stemmed_words = [stemmer.stem(w) for w in words]
    stemmed_phrase = " ".join(stemmed_words)

    return stemmed_phrase

tqdm.pandas(desc="stemming...")
train["Phrase(Stemmed)"] = train["Phrase"].progress_apply(stem_phrase)

train.head()


stemming...:  54%|████████████████████████████████▎                           | 84146/156060 [00:14<00:16, 4469.32it/s]


stemming...: 100%|███████████████████████████████████████████████████████████| 156060/156060 [00:27<00:00, 5686.31it/s]

,SentenceId,Phrase,Sentiment,Phrase(Origin),Phrase(Stemmed)
PhraseId,,,,,
1,1,A series of escapades demonstrating the adage ...,1,A series of escapades demonstrating the adage ...,a seri of escapad demonstr the adag that what ...
2,1,A series of escapades demonstrating the adage ...,2,A series of escapades demonstrating the adage ...,a seri of escapad demonstr the adag that what ...
3,1,A series,2,A series,a seri
4,1,A,2,A,a
5,1,series,2,series,seri


# Lemmatizer

In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer= WordNetLemmatizer(); lemmatizer

def clean_phrase_lemmatizer(phrase):
    words = phrase.split(' ')
    lemmatized_words = [lemmatizer.lemmatize(w) for w in words]
    lemmatized_phrase = ' '.join(lemmatized_words)
    return lemmatized_phrase

#train에 적용
train["Phrase"] = train["Phrase"].apply(clean_phrase_lemmatizer)

# Pos tagger

In [95]:
nltk.download('punkt')
from nltk import pos_tag
from nltk.tokenize import word_tokenize # pos tagger 전용
phrase= train.loc[2274]["Phrase"]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\optim\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [96]:
words_pos= pos_tag(word_tokenize(phrase))
words_pos

phrase= "you should close the door"
words= phrase.split(" ")

stemmed_words= [stemmer.stem(w) for w in words]
stemmed_phrase= " ".join(stemmed_words)

pos_tag(word_tokenize(stemmed_phrase))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\optim\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[('you', 'PRP'),
 ('should', 'MD'),
 ('close', 'VB'),
 ('the', 'DT'),
 ('door', 'NN')]

### Clean Text

In [33]:
def clean_text(phrase):
    phrase = phrase.replace("ca n't", "can not")
    phrase = phrase.replace("does n't", "does not")
    phrase = phrase.replace("n't", "not")

    return phrase

In [34]:
train["Phrase"] = train["Phrase"].apply(clean_text)

print(train.shape)
train[["Phrase", "Phrase(Origin)"]].head()

(156060, 4)


,Phrase,Phrase(Origin)
PhraseId,,
1,A series of escapades demonstrating the adage ...,A series of escapades demonstrating the adage ...
2,A series of escapades demonstrating the adage ...,A series of escapades demonstrating the adage ...
3,A series,A series
4,A,A
5,series,series


In [35]:
test["Phrase"] = test["Phrase"].apply(clean_text)

print(test.shape)
test[["Phrase", "Phrase(Origin)"]].head()

(66292, 3)


,Phrase,Phrase(Origin)
PhraseId,,
156061,An intermittently pleasing but mostly routine ...,An intermittently pleasing but mostly routine ...
156062,An intermittently pleasing but mostly routine ...,An intermittently pleasing but mostly routine ...
156063,An,An
156064,intermittently pleasing but mostly routine effort,intermittently pleasing but mostly routine effort
156065,intermittently pleasing but mostly routine,intermittently pleasing but mostly routine


### Vectorize Phrases

In [72]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=10000, ngram_range=(1, 2))

vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=10000, min_df=1,
        ngram_range=(1, 5), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [73]:
vectorizer.fit(train["Phrase"])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=10000, min_df=1,
        ngram_range=(1, 5), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [74]:
X_train = vectorizer.transform(train["Phrase"])
X_train

<156060x10000 sparse matrix of type '<class 'numpy.int64'>'
	with 1169502 stored elements in Compressed Sparse Row format>

In [75]:
vocabulary = vectorizer.get_feature_names()
pd.DataFrame(X_train[0:100].toarray(), columns=vocabulary).head()

,10,10 minutes,10 year,100,101,11,12,12 year,12 year old,13,...,your time,your watch,yourself,youth,youthful,zeal,zero,zhang,zombie,zone
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [76]:
X_test = vectorizer.transform(test["Phrase"])
X_test

<66292x10000 sparse matrix of type '<class 'numpy.int64'>'
	with 426657 stored elements in Compressed Sparse Row format>

In [77]:
y_train = train["Sentiment"]

print(y_train.shape)
y_train.head()

(156060,)


PhraseId
1    1
2    2
3    2
4    2
5    2
Name: Sentiment, dtype: int64

### Train

In [78]:
from sklearn.linear_model import SGDClassifier

model = SGDClassifier(random_state=37)
model

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=37, shuffle=True,
       tol=None, verbose=0, warm_start=False)

### Scoring

In [79]:
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GroupKFold

kfold = GroupKFold(n_splits=5)

y_predict = cross_val_predict(model, X_train, y_train,
                              cv=kfold, groups=train["SentenceId"])

print(y_predict.shape)
y_predict

c:\program files\python36\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
c:\program files\python36\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
c:\program files\python36\lib\

(156060,)


array([3, 3, 2, ..., 2, 2, 2], dtype=int64)

In [80]:
from sklearn.metrics import accuracy_score

score = accuracy_score(y_train, y_predict)

print("Score = {0:0.6f}".format(score))

Score = 0.583385


### score init: 0.583602 (countVectorize: 1,2 가 max)

In [81]:
import numpy as np

result = train.copy()
result["Sentiment(predict)"] = y_predict

result["Distance"] = result["Sentiment"] - result["Sentiment(predict)"]
result["Distance"] = np.abs(result["Distance"])

result = result.sort_values(by="Distance", ascending=False)
result.head()

,SentenceId,Phrase,Sentiment,Phrase(Origin),Sentiment(predict),Distance
PhraseId,,,,,,
14964,643,can such a cold movie claim to express warmth ...,0,can such a cold movie claim to express warmth ...,4,4
124187,6667,joy to watch and -- especially -- to listen to,4,joy to watch and -- especially -- to listen to,0,4
48041,2343,would be a shame if this was your introduction...,0,would be a shame if this was your introduction...,4,4
48040,2343,it would be a shame if this was your introduct...,0,it would be a shame if this was your introduct...,4,4
61625,3115,There is no substitute for on-screen chemistry...,4,There is no substitute for on-screen chemistry...,0,4


In [82]:
# result[result["Phrase"].str.contains("can't")]

In [83]:
result[0:10000].to_csv("result.csv")

In [84]:
pd.DataFrame(vocabulary).to_csv("vocabulary.csv")

### Predict

In [85]:
model.fit(X_train, y_train)

c:\program files\python36\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=37, shuffle=True,
       tol=None, verbose=0, warm_start=False)

In [86]:
predictions = model.predict(X_test)

print(predictions.shape)
predictions

(66292,)


array([3, 3, 2, ..., 2, 2, 2], dtype=int64)

## Submit

In [87]:
submission = pd.read_csv("data/sampleSubmission.csv", index_col="PhraseId")

print(submission.shape)
submission.head()

(66292, 1)


,Sentiment
PhraseId,
156061,2
156062,2
156063,2
156064,2
156065,2


In [88]:
submission["Sentiment"] = predictions

print(submission.shape)
submission.head()

(66292, 1)


,Sentiment
PhraseId,
156061,3
156062,3
156063,2
156064,3
156065,3


In [89]:
# submission.to_csv("baseline-script.csv")